In [1]:
import pandas as pd

In [2]:
# Format of numbers are in decimal format
pd.options.display.float_format = '{:.2f}'.format

In [3]:
# Import data
df = pd.read_csv("fhv_data.csv", index_col=[0])
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02872,B02872,2020-01-01 05:24:21,2020-01-01 05:25:37,2020-01-01 05:26:11,2020-01-01 05:43:33,24,127,6.06,...,2.05,0.00,NaN,0.00,15.23,N,N,,N,N
1,HV0003,B02876,B02876,2020-01-05 18:24:51,2020-01-05 18:27:38,2020-01-05 18:29:58,2020-01-05 18:39:32,87,261,1.30,...,0.82,2.75,NaN,0.00,7.20,N,N,,N,N
2,HV0003,B02764,B02764,2020-01-02 17:12:44,2020-01-02 17:14:11,2020-01-02 17:15:12,2020-01-02 17:31:48,77,63,2.73,...,0.84,0.00,NaN,0.00,11.26,N,N,,N,N
3,HV0003,B02764,B02764,2020-01-20 18:43:02,2020-01-20 18:43:15,2020-01-20 18:44:53,2020-01-20 18:53:57,21,108,1.72,...,1.00,0.00,NaN,0.00,6.37,N,N,,N,N
4,HV0003,B02871,B02871,2020-01-19 23:16:43,2020-01-19 23:26:49,2020-01-19 23:27:50,2020-01-19 23:52:02,36,188,5.05,...,1.47,0.00,NaN,0.00,11.29,Y,Y,,N,N


In [4]:
# Replace pick up and drop off ids with zone and borough names
taxi_lookup = pd.read_csv("taxi_zone_lookup.csv")
taxi_lookup = taxi_lookup[["LocationID", "Zone", "Borough"]].set_index("LocationID")
zone_lookup = taxi_lookup[["Zone"]].to_dict()["Zone"]
borough_lookup = taxi_lookup[["Borough"]].to_dict()["Borough"]

In [5]:
# Store the zone and boroughs in the dataframe
df["pu_zone"] = df["PULocationID"].map(zone_lookup)
df["pu_borough"] = df["PULocationID"].map(borough_lookup)
df["do_zone"] = df["DOLocationID"].map(zone_lookup)
df["do_borough"] = df["DOLocationID"].map(borough_lookup)

In [6]:
# Drop the IDs
df.drop(columns=["PULocationID", "DOLocationID"], inplace=True)

In [7]:
# Map ride share companies
df["rideshare_company"] = df["hvfhs_license_num"].replace(
    to_replace=["HV0002", "HV0003", "HV0004", "HV0005"],
    value=["Juno", "Uber", "Via", "Lyft"]
    )

In [8]:
# Drop extra columns
df.drop(columns=["hvfhs_license_num", "dispatching_base_num", "originating_base_num"], inplace=True)

In [9]:
# Get info on the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30500000 entries, 0 to 30499999
Data columns (total 24 columns):
 #   Column                Dtype  
---  ------                -----  
 0   request_datetime      object 
 1   on_scene_datetime     object 
 2   pickup_datetime       object 
 3   dropoff_datetime      object 
 4   trip_miles            float64
 5   trip_time             int64  
 6   base_passenger_fare   float64
 7   tolls                 float64
 8   bcf                   float64
 9   sales_tax             float64
 10  congestion_surcharge  float64
 11  airport_fee           float64
 12  tips                  float64
 13  driver_pay            float64
 14  shared_request_flag   object 
 15  shared_match_flag     object 
 16  access_a_ride_flag    object 
 17  wav_request_flag      object 
 18  wav_match_flag        object 
 19  pu_zone               object 
 20  pu_borough            object 
 21  do_zone               object 
 22  do_borough            object 
 23  rideshare_

In [10]:
# Convert objects to datetimes
df["request_datetime"] = pd.to_datetime(df["request_datetime"], format="mixed", errors="coerce")
df["on_scene_datetime"] = pd.to_datetime(df["on_scene_datetime"], format="mixed", errors="coerce")
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], format="mixed", errors="coerce")
df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"], format="mixed", errors="coerce")

In [11]:
# Rideshare companies and later filter it to only Uber and Lyft
df["rideshare_company"].value_counts()

rideshare_company
Uber    22168007
Lyft     8191827
Via       140166
Name: count, dtype: int64

In [12]:
df = df[(df["rideshare_company"] == "Uber")]
df.reset_index(drop=True, inplace=True)
df.shape

(22168007, 24)

In [13]:
# Check for empty values
df.isna().sum()

request_datetime              0
on_scene_datetime             0
pickup_datetime               0
dropoff_datetime              0
trip_miles                    0
trip_time                     0
base_passenger_fare           0
tolls                         0
bcf                           0
sales_tax                     0
congestion_surcharge          0
airport_fee             5358403
tips                          0
driver_pay                    0
shared_request_flag           0
shared_match_flag             0
access_a_ride_flag            0
wav_request_flag              0
wav_match_flag                0
pu_zone                       0
pu_borough                  979
do_zone                       0
do_borough               823578
rideshare_company             0
dtype: int64

In [14]:
# Drop any rows that are empty 
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [15]:
# Double check
df.isna().sum()

request_datetime        0
on_scene_datetime       0
pickup_datetime         0
dropoff_datetime        0
trip_miles              0
trip_time               0
base_passenger_fare     0
tolls                   0
bcf                     0
sales_tax               0
congestion_surcharge    0
airport_fee             0
tips                    0
driver_pay              0
shared_request_flag     0
shared_match_flag       0
access_a_ride_flag      0
wav_request_flag        0
wav_match_flag          0
pu_zone                 0
pu_borough              0
do_zone                 0
do_borough              0
rideshare_company       0
dtype: int64

In [16]:
# Check for duplicates
df.duplicated().sum()

np.int64(0)

In [17]:
# Remove records with base bassenger fare or driver pay less than or equal to $0 and driver pay more than $350
df = df[~((df["base_passenger_fare"] <= 0) | (df["driver_pay"] <= 0) | (350 <= df["driver_pay"]))]
df.reset_index(drop=True, inplace=True)
df.shape

(16072312, 24)

In [18]:
# Calculate the percentage of base passenger fare paid to drivers
df["perct_of_base_fare"] = df["driver_pay"]/df["base_passenger_fare"]

# Filter the data to remove payments less than 25% and greater than 100% (Lyft keeps about 30%)
df = df[(df["perct_of_base_fare"] <= 1.00) & (0.25 <= df["perct_of_base_fare"])]
df.reset_index(drop=True, inplace=True)
df.shape

(13880691, 25)

In [19]:
# Export the data to a .csv file
df.to_csv("fhv_data_fixed.csv")